In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
from data_loader import prepare_ml_pipeline, MovieLensData, MovieLensDataset
from matrix_factor import BiasedMF, train_mf
from debiasing import train_debiasing_model, DebiasingModel
from dropoutnet import DeepCF, train_dropoutnet
from evaluator import ndcg_calc_base, ndcg_calc_dropout, ndcg_calc_debiased, ndcg_calc_sampled, ndcg_calc_dropout_sampled, ndcg_calc_debiased_sampled



In [ ]:
def run_evaluation_pipeline(device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):
    """Complete evaluation pipeline with DropoutNet and debiasing"""
    
    # Define evaluation parameters
    k_values = [15,30]
    

    dropoutnet_params = {
        'model_select': [800, 400],
        'rank_out': 200,
        'dropout_rate': 0.5,
        'batch_size': 1000,
        'n_scores_per_user': 2500,
        'data_batch_size': 1000,
        'max_data_per_step': 50000,
        'num_epochs': 1,
        'learning_rate': 0.001
    }
    
    #Params from https://github.com/Zziwei/Fairness-in-Cold-Start-Recommendation/blob/main/Scale/main.py
    debiasing_params = {
        'model_select': [100],
        'alpha': 4.0,
        'batch_size': 50,
        'num_epochs': 1,
        'reg': 0.00001
    }
    
    # 1. Load and prepare data
    print("Loading data...")
    ml_data, train_loader, valid_loader, test_loader = prepare_ml_pipeline(cold_start=True)
    #evaluator = RecommenderEvaluator(ml_data)
    
    # 2. Train and evaluate base model
    print("\nTraining base model...")
    base_model = BiasedMF(ml_data.n_users, ml_data.n_items).to(device)
    base_model = train_mf(model = base_model, train_loader= train_loader, val_loader= valid_loader, ml_data= ml_data, num_epochs=1, lr = .01)
    
    base_ndcgs, base_prec, base_recall, final_mdg, mdg_anal = ndcg_calc_base(base_model, test_loader, ml_data, k_values=k_values)
    print(f"Base NDCGS: {base_ndcgs}")
    #print(f"Final MDG: {mdg_anal['all']['mean']}, min10: {mdg_anal['bottom_10']['mean']}, min20: {mdg_anal['bottom_20']['mean']}, top10: {mdg_anal['top_10']['mean']}")

    

    
    #3. Train and evaluate DropoutNet model
    print("\nTraining DropoutNet model...")
    dropoutnet = train_dropoutnet(
        ml_data=ml_data,
        base_model=base_model,
        val_loader=valid_loader,
        test_loader=test_loader,
        **dropoutnet_params,
        device=device
    )

    print("\nEvaluating DropoutNet model...")
    dropout_ndcgs, drop_mdg, drop_mdg_anal = ndcg_calc_dropout_sampled(base_model, dropoutnet, test_loader, ml_data, k_values = k_values)
    print(f"Dropout NDCGs {dropout_ndcgs}")
    print(f"Final MDG: {drop_mdg_anal['all']['mean']}, min10: {drop_mdg_anal['bottom_10']['mean']}, min20: {drop_mdg_anal['bottom_20']['mean']}, top10: {drop_mdg_anal['top_10']['mean']}")

    
    # 4. Train and evaluate debiasing model
    print("\nTraining debiasing model...")
    debiasing_model = train_debiasing_model(
        base_model=dropoutnet,
        original_mf=base_model,
        ml_data=ml_data,
        **debiasing_params,
        device=device
    )
    
    print("\nEvaluating debiased model...")
    debiased_ndcgs, debiased_prec, debiased_rec, debiased_mdg, debiased_mdg_anal = ndcg_calc_debiased_sampled(dropoutnet, base_model, debiasing_model,test_loader,ml_data,k_values=k_values)
    
    print(f"Debiased NDCGs {debiased_ndcgs}")
    print(f"Final MDG: {debiased_mdg_anal['all']['mean']}, min10: {debiased_mdg_anal['bottom_10']['mean']}, min20: {debiased_mdg_anal['bottom_20']['mean']}, top10: {debiased_mdg_anal['top_10']['mean']}")
    
   
    return 0

In [3]:
test = run_evaluation_pipeline()

Loading data...
Dataset loaded with cold_start=True:
Train: 692958 interactions
Valid: 94109 interactions
Test: 213142 interactions
Cold-start statistics:
Valid items not in train: 370
Test items not in train: 741

Training base model...
Epoch 1 - Avg Loss: 1.7364 - Avg Train NDCG: 0.6936 - Avg Test NDCG: 0.0148
Epoch 1 - Train Precision: 0.0000 - Train Recall: 0.0000 - Avg Test Prec: 0.0092 - Avg Tet Rec: 0.023722199298856106
Base NDCGS: [0.7241947991582373, 0.7692262617376064]

Training DropoutNet model...
Initializing DropoutNet training...
	u_concat rank=100
	v_concat rank=118
Starting training for 1 epochs...


/Users/jackklawitter/Documents/Rutgers/FairAI/FairColdRec/dropoutnet.py:590: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Uin = torch.tensor(u_emb_expanded[batch_u_idx], device=device)
/Users/jackklawitter/Documents/Rutgers/FairAI/FairColdRec/dropoutnet.py:591: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Vin = torch.tensor(i_emb_expanded[batch_i_idx], device=device)


Step 100: Loss = 0.0092
Step 200: Loss = 0.0086
Step 300: Loss = 0.0081
Epoch 1/1: Average Loss = 0.0132

Training completed!

Evaluating DropoutNet model...


TypeError: ndcg_calc_dropout_sampled() got an unexpected keyword argument 'ks'